This files contains both implementation and report of question 2.

Importing libraries

In [1]:
import numpy as np

Initializing train and test matrices

In [2]:
Et = np.zeros((5, 16))
Es = np.zeros((1, 16))

Loading train and test samples<br>
STDID = 401722199<br>
9 % 3 = 0 ==> set 0<br>

In [3]:
Et[0, :] = [0,1,1,0, 1,0,0,1, 1,1,1,1 ,1,0,0,1] # A
Et[1, :] = [0,1,1,1, 1,0,0,0, 1,0,0,0 ,0,1,1,1] # C
Et[2, :] = [0,1,1,1, 0,0,1,0, 1,0,1,0 ,1,1,1,0] # J
Et[3, :] = [1,0,0,0, 1,0,0,0, 1,0,0,0, 1,1,1,1] # L
Es[0, :] = [0,1,1,0, 1,0,0,1, 1,0,0,1, 0,1,1,0] # O

names = {0:'A',1:'C',2:'J',3:'L',4:'O'}

This impelementation of ART algorithm is based on the following github imp:<br>
https://github.com/rougier/neural-networks/blob/master/art1.py <br>
I did not find any better imp.

In [4]:
class ART:
    def __init__(self, Et, n, rho=0.1):
        m,f = Et.shape
        # Comparison layer: it has the same number of features as inputs
        self.F1 = np.ones(f) 
        # Recognition layer: it has the same number of features as the maximum number of classes (n)
        self.F2 = np.ones(n)
        #### Between F1 and F2 there is a fully bidirectional network, so Wf is the forward weights and Wb is the backward weights.
        # Feed-forward weights 
        self.Wf = np.random.random((n,f))
        # Feed-back weights
        self.Wb = np.random.random((f,n))
        # Vigilance
        self.rho = rho
        # Number of active units in F2
        self.active = 0


    def learn(self, X):
        # Compute F2 output and sort them (I)
        # We feed the input sample X through the forward direction of netowrk to calculated F2 values
        self.F2[...] = np.dot(self.Wf, X)
        # Sort values
        I = np.argsort(self.F2[:self.active].ravel())[::-1]

        for i in I:
            # Check if nearest memory is above the vigilance level
            d = (self.Wb[:,i]*X).sum()/X.sum()
            if d >= self.rho:
                # Learn data
                self.Wb[:,i] *= X
                self.Wf[i,:] = self.Wb[:,i]/(0.5+self.Wb[:,i].sum())
                return self.Wb[:,i], i

        # No match found, increase the number of active units
        # and make the newly active unit to learn data
        if self.active < self.F2.size:
            i = self.active
            self.Wb[:,i] *= X
            self.Wf[i,:] = self.Wb[:,i]/(0.5+self.Wb[:,i].sum())
            self.active += 1
            return self.Wb[:,i], i

        return None,None

We trained multiple networks with different rhos. <br>
Then we test the network on training set and test set with different rho values <br>
As we increase the value of rho, we can see that the number of prototypes increases and this means that the similarity test between input and prototypes was so hard that network created a new prototype. So increasing the rho, will cause an increament to number of prototypes and after a certain threshold, it cause grandmother node effect.

In [5]:
for i in range(5, 36, 5): # testing different rho values.
    rho = i / 100 # converting i between 0 to 1
    print(f'RHO: {rho}')
    net = ART(Et, 5, rho)  # initing the ART1 network
    dic_prev_prototype = {} # Previous iter weights
    while True:
        dic_current_prototype = {} # Current iter weights
        for i in range(4): # learn each input
                prototype, k = net.learn(Et[i, :]) # learns this sample and returns its corresponding prototype and class
                dic_current_prototype[k] = prototype # store this prototype for further convergance tests

        a = set(dic_prev_prototype.keys()) # get a list of prvious classes
        b = set(dic_current_prototype.keys()) # get a list of new iter classes
        do_cont = True
        if len(b.difference(a)) != 0: # if there is a new class, continue the learning
            dic_prev_prototype = dic_current_prototype
        else: # if classes of this iter is the same as before, check the prototypes and find if there is differnce there
            do_cont = False
            for key in a:
                if np.array_equal(dic_prev_prototype[key], dic_current_prototype[key]):
                    continue
                else:
                    do_cont = True
                    break
            dic_prev_prototype = dic_current_prototype
        if not do_cont:
            break

    for k in sorted(dic_current_prototype.keys()): # print the final prototype vectors
        print(f'Class {k} prototype: {list(dic_current_prototype[k])}')
    # try to classify train cases again (after convergance)
    for i in range(4):
        _, k = net.learn(Et[i,:])
        print(f'{names[i]}-> class {k}')

    # try to classify test case
    prototype, k = net.learn(Es[0, :])
    print(f'{names[4]}-> class {k}')
    if k not in dic_current_prototype.keys():
        print(f'Test case was not similar to current prototypes: \n New Prototype: {prototype}')
    print('-'*40)



RHO: 0.05
Class 0 prototype: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.530842746904706, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
A-> class 0
C-> class 0
J-> class 0
L-> class 0
O-> class 0
----------------------------------------
RHO: 0.1
Class 0 prototype: [0.0, 0.9395948429151861, 0.041667694533356614, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08034933234817987, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Class 1 prototype: [0.303478554941387, 0.0, 0.0, 0.0, 0.7620603073699114, 0.0, 0.0, 0.0, 0.1629801749361478, 0.0, 0.0, 0.0, 0.9908744649972181, 0.6603189870062347, 0.7436957967684719, 0.4131072096141828]
A-> class 0
C-> class 0
J-> class 0
L-> class 1
O-> class 0
----------------------------------------
RHO: 0.15
Class 0 prototype: [0.0, 0.0, 0.0, 0.0, 0.9687280475048475, 0.0, 0.0, 0.0, 0.06914318270868014, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9401740532524722]
Class 1 prototype: [0.0, 0.5460755455212253, 0.24615012694587135, 0.1510475888899817, 0.0, 0.0, 0.07671888022086026, 0.0, 0.601255480866512, 0.0, 0.792992999